In [270]:
%matplotlib inline
import urllib, time, json, csv
import numpy as np
import pandas as pd

In [28]:
write_path = 'auctions.json'
#api_key = 'CharlesZ-11ce-4517-8df3-becc92494006'
api_key = 'MichaelA-757e-4b18-8eeb-2a0a915831f3'

The following three functions are what we are using to make api calls. These calls are through a url link, which we are using the library urllib to get the response data from. This response data is in the form of a json, which we convert to a dictionary using the json library.

In [271]:
# for finding api, findCompletedItems calls
def finding_data(page):
    # parameter and values for call
    paramKeys = ['OPERATION-NAME', 'SERVICE-VERSION','SECURITY-APPNAME','RESPONSE-DATA-FORMAT','categoryId', 'paginationInput.pageNumber', "keywords"]
    paramVals = ['findCompletedItems','1.7.0',api_key,'JSON&REST-PAYLOAD','9355', str(page), "iphone+5s+32GB"]

    # call url
    url_finding = 'http://svcs.ebay.com/services/search/FindingService/v1?'
    
    # constructing the url
    url = url_finding
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    
    # loading the return response
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    # parse data for selected auctions
    return data['findCompletedItemsResponse'][0]['searchResult'][0]['item']



In [30]:
# for shopping api, getSingleItem calls
def item_data(item_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseencoding', 'appid', 'siteid', 'version', 'ItemID', 'IncludeSelector']
    paramVals = ['GetSingleItem', 'JSON', api_key, '0', '515', item_id, 'Details']
    
    # call url
    url_item = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_item
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
        
    # loading the return response, keep trying until success
    while True:
        try:
            response = urllib.urlopen(url)
            data = json.loads(response.read())
            break
        except ValueError:
            #time.sleep(1)
            pass
        
    
    #parse data for selected auction
    return data['Item']


In [33]:
# for shopping api, getUserProfile calls
def user_data(user_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseseencoding', 'appid', 'sideid', 'version', 'UserID', 'IncludeSelector']
    paramVals = ['GetUserProfile', 'JSON', api_key, '0', '525', user_id, 'FeedbackHistory']
    
    # call url
    url_user = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_user
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    
    # loading the return response 
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    #parse data for selected auction
    return data

Here we are grabbing data from Ebay's Finding API. The maximum number of calls we can make is 10000, but we also need to filter out duplicates (which sometimes occur). We then 

In [274]:
# finding api
# loop backwards to avoid conflicts
%time
repeat_set = set()
nodup_totaldata = []
for i in range(0,100):
    data = finding_data(100-i)
    time.sleep(1) 
    for auc in data:
        if auc['itemId'][0] not in repeat_set:
            repeat_set.add(auc['itemId'][0])
            nodup_totaldata.append(auc)
    


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


Create a new file and dump this data into it.

In [297]:
repeat_set = set()
for auc in nodup_totaldata:
    repeat_set.add(auc['itemId'][0])
print len(repeat_set), len(nodup_totaldata)

9659 9659


In [292]:
with open('completedItems.json', 'w') as outfile:
    json.dump(nodup_totaldata, outfile)

Calling getSingleItem from Ebay's Shopping API, this function takes some time because we are making over 9000 calls, and sometimes the calls fail, which means we need to remake the call.

In [298]:
%%time
auc_data = []
removed_set = set()
counter = 0
for auc in list(repeat_set):
    try:
        data = item_data(auc)
        auc_data.append(data)
        counter +=1
        print counter, "auctions retrieved"
    except KeyError:
        # auction id doesn't exist? remove auction from set
        removed_set.add(auc)
        repeat_set.remove(auc)
        #continue
        continue
    
    


1 auctions retrieved
2 auctions retrieved
3 auctions retrieved
4 auctions retrieved
5 auctions retrieved
6 auctions retrieved
7 auctions retrieved
8 auctions retrieved
9 auctions retrieved
10 auctions retrieved
11 auctions retrieved
12 auctions retrieved
13 auctions retrieved
14 auctions retrieved
15 auctions retrieved
16 auctions retrieved
17 auctions retrieved
18 auctions retrieved
19 auctions retrieved
20 auctions retrieved
21 auctions retrieved
22 auctions retrieved
23 auctions retrieved
24 auctions retrieved
25 auctions retrieved
26 auctions retrieved
27 auctions retrieved
28 auctions retrieved
29 auctions retrieved
30 auctions retrieved
31 auctions retrieved
32 auctions retrieved
33 auctions retrieved
34 auctions retrieved
35 auctions retrieved
36 auctions retrieved
37 auctions retrieved
38 auctions retrieved
39 auctions retrieved
40 auctions retrieved
41 auctions retrieved
42 auctions retrieved
43 auctions retrieved
44 auctions retrieved
45 auctions retrieved
46 auctions retriev

In [299]:
json.dump(total_auc_data,outfile)
outfile.close()

ValueError: I/O operation on closed file

In [238]:
user_ids = set()
for x in total_auc_data:
    user_ids.add(x['Seller']['UserID'])
    


In [43]:
# user_list = []
# with open('auctions_detailed.json') as testfile:
#     reader = json.load(testfile)
#     for auct in reader:
#         user_list += auct['Seller']['UserID']
#         print auct['Seller']['UserID']
count=0
userlist = []
user_set = set()
with open('auctions_detailed.json') as testfile:
    reader = json.load(testfile)
    for auct in reader:
        username = auct['Seller']['UserID']
        print username
        count+=1
        userlist.append(username)
#     for auct in reader:
#         user_set.add(auct['Seller']['UserID'])
#         print auct['Seller']['UserID']
print len(userlist)

abtboss
jgmcompsupport
flappy_deal
chocolatecakemu
bstew2525
erlamar2011
domcia_w
dres-andr
berlioth1
huoikt88
wallachee
wallachee
pende_kath
dwrinc
www_cent
vamudder
wise-deals
joagerme0
annearundeljewelrybuyers
sparkles2119
upgradesolution
gazelle-store
rodriguezmujica
barnyardwireless
us2015.fos
toniesoprano
gooverstock
fulllove365
moshe2908
alex1855588
upgradesolution
v7stars
juligate
dylankays
davie-deals
sasha_il
jwiedenman
alloutmobil
partsonlyphones
alex1855588
theepp
mstoyzz
alltechwholesale
wine438
goldpony72
techmiaonline
1theonly
nodzade
twinmom505
sellercortex
alltechwholesale
pawns-not-redeemed
denni.arace
celldays
trimman10
thermokool
wessterling22
shutterblade
ritchmobiles
ajseattle
khangoodguy_228
robin-9002i0
2014happyhong
tonkyota
ecellsolutions
shutterblade
mcchronic_276
mwaura16
antoinettenoz
discountedcelltech
sellingtech13
nbmc299
dwrinc
toyboy321
paubo52
ducklady_101
dinocubs1
thephonguy
ltinius1988
ccclubmis
erika4b
jerryp76
jerryp76
relertech3
fargo10101
gmmad

In [24]:
len(user_set)

5922

In [45]:
len(set(userlist))

5922

In [246]:
counter = 0
user_data = []
for user in list(user_ids):
    data = user_data(user)
    user_data.append(data)

len(user_data)

TypeError: 'list' object is not callable

In [250]:
with open('auctions.json') as fd:
    reader = json.load(fd)
    print len(reader)


20000


In [251]:
len(total_auc_data)

9856